In [ ]:
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from api_util import read_token, BASE_URL, get_headers
import requests


token = read_token()

def _make_get_request(endpoint, params=None, quiet=None):
    """共用的 HTTP GET 請求處理函數"""
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=get_headers(token), params=params)
        response.raise_for_status()
        result = response.json()
        if not quiet: 
            print(result)
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None
    
def _make_post_request(endpoint, data, quiet=None):
	try:
		response = requests.post(f"{BASE_URL}/{endpoint}", headers=get_headers(token), json=data)
		response.raise_for_status()
		result = response.json()
		if not quiet: 
			print(result)
		return result
	except requests.exceptions.RequestException as err:
		print(f"Request failed: {err}")
		return None
    
def get_all_technews (page, size, keyword=None):
	res =  _make_get_request('technews',  params={
		'page': page,
		'size': size,
        'keyword': keyword
	})
	return res.get('data')[::-1]
    
def create_one_company_news (params):
    res = _make_post_request('/company-news', params)
    return res
    
def get_all_company_news (page, size):
	res =  _make_get_request('company-news', {
		'page': page,
		'size': size
	})
	return res.get('data')[::-1]
    
def get_all_company_news_by_keyword (keyword, page=1, size=10):
	res =  _make_get_request('/company-news/search', {
		'page': page,
		'size': size,
		'keyword': keyword
	})
	return res.get('data')

def get_user_favorite_news():
    res = _make_get_request('userFavorite/technews')
    return res.get('data')
    
def user_subscribe_news(params):
    res = _make_post_request('/userFavorite/technews', params)
    return res

def print_news(news):
	for item in news:
		id = item.get('id', '--')
		title = item.get('title', '--')
		print(f"{id} : {title} \n")

In [ ]:
keyword = 'ai'

news = get_all_technews(1, 10, keyword=keyword)

print_news(news)

In [ ]:
all_news = get_all_technews(page=1, size=100)
print_news(all_news)

In [ ]:
all_news = get_all_company_news(page=1, size=5)

print_news(all_news)

In [35]:
keyword = 'ai'
search_news = get_all_company_news_by_keyword(page=1, size=5, keyword=keyword)

print_news(search_news)

{'code': 200, 'message': '成功', 'data': [{'id': 4950, 'title': 'Analyst who nailed Teslas collapse has a new prediction as Musk turns bearish', 'company_id': 117, 'release_time': '2025-06-06T03:19:00.000Z', 'publisher': 'TheStreet', 'web_url': 'https://finance.yahoo.com/m/6623c151-278e-35b0-9419-e243a29d2f9d/analyst-who-nailed-tesla%E2%80%99s.html', 'createdAt': '2025-06-06T03:32:51.000Z', 'updatedAt': '2025-06-06T03:32:51.000Z', 'Company': {'name': 'Tesla, Inc.'}}, {'id': 4918, 'title': 'Commentary: Elon Musk aims his chainsaw at Congress  and Trump', 'company_id': 117, 'release_time': '2025-06-06T07:13:00.000Z', 'publisher': 'Yahoo Finance', 'web_url': 'https://finance.yahoo.com/news/commentary-elon-musk-aims-his-chainsaw-at-congress--and-trump-191316582.html', 'createdAt': '2025-06-06T03:32:47.000Z', 'updatedAt': '2025-06-06T03:32:47.000Z', 'Company': {'name': 'Tesla, Inc.'}}, {'id': 4914, 'title': 'Tesla shares on pace for worst daily drop in almost five years', 'company_id': 117, '

In [ ]:
symbol = 'tsla'

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone

with open('index.html', 'r', encoding='utf-8') as file:
	html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

news = []

for row in soup.find_all('tr', class_='cursor-pointer has-label'):
	time_cell = row.find('td', align='right')
	time_text = time_cell.get_text(strip=True).replace('Today', '')

	if 'Today' in time_text:
		time_text = time_text.replace('Today', '')
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()
	else:
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()

	try:
		time_obj = datetime.strptime(time_text, '%I:%M%p')
		time_24hr = time_obj.strftime('%H:%M:%S')
	except ValueError:
		# 如果时间格式无法解析，使用默认时间
		time_24hr = '00:00:00'
		
	# 创建 MySQL 支持的时间字符串
	release_time = f"{date_str} {time_24hr}"

	link = row.find('a', class_='tab-link-news')
	title = link.get_text(strip=True)
	web_url = link['href']

	publisher = row.find('span').get_text(strip=True).strip('()')

	news_item = {
        "title": title,
        "symbol": symbol.upper(),
        "release_time": release_time,
        "publisher": publisher,
        "web_url": web_url
    }

	news.append(news_item)

print(json.dumps(news[0:2], indent=4))


In [ ]:
from time import sleep

SLEEP_SECOND = 1/ 10
for item in news:
	res = create_one_company_news(item)
	print(res)
	sleep(SLEEP_SECOND)

In [ ]:
newsId = 123262
data = user_subscribe_news({'newsId': newsId})
data

In [ ]:
data = get_user_favorite_news()
data